In [4]:
import utils

audioFileOutput = "audio_ouput.mp3"
text = "Hello World! Hear me speak!"

audioFileInput = "input4.wav" 

utils.setupOpenAIAPI()
utils.textToAudioFile(text, audioFileOutput) # Convert example text to audio file as test

# transcription = utils.audioToText(audioFileInput)
# print(transcription)

In [ ]:
import os
import utils  # Make sure you have a utils.py file for your OpenAI API interactions
import sounddevice as sd
import scipy.io.wavfile as wav
import numpy as np
import webrtcvad
from pydub import AudioSegment
from pydub.playback import play
import collections
import time

# Initialize OpenAI API 
# (replace with your actual API key setup from utils.py)
utils.setupOpenAIAPI() 

# Define audio parameters
fs = 16000 
frame_duration_ms = 30 

def record_audio_vad(filename):
    vad = webrtcvad.Vad(1)
    num_padding_frames = 10 
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    triggered = False
    voiced_frames = []
    print("Recording... Speak now.")

    silence_threshold_seconds = 1.5 
    last_voice_timestamp = time.time()

    with sd.InputStream(samplerate=fs, channels=1, dtype='int16') as stream:
        while True:
            audio_chunk, overflowed = stream.read(int(fs * frame_duration_ms / 1000))
            is_speech = vad.is_speech(audio_chunk.tobytes(), fs)

            if is_speech:
                last_voice_timestamp = time.time()

            if not triggered:
                ring_buffer.append((audio_chunk, is_speech))
                num_voiced = len([f for f, speech in ring_buffer if speech])
                if num_voiced > 0.9 * ring_buffer.maxlen:
                    triggered = True
                    print("Recording started.")
                    voiced_frames.extend([f for f, s in ring_buffer])
                    ring_buffer.clear()
            else:
                voiced_frames.append(audio_chunk)
                ring_buffer.append((audio_chunk, is_speech))
                num_unvoiced = len([f for f, speech in ring_buffer if not speech])
                
                if num_unvoiced > 0.9 * (ring_buffer.maxlen) and (time.time() - last_voice_timestamp > silence_threshold_seconds):
                    print("Recording stopped.")
                    break

    audio_data = np.concatenate(voiced_frames, axis=0)
    wav.write(filename, fs, audio_data)
    print("Recording complete.")

def transcribe_audio(filename):
    # Replace with your actual transcription logic from utils.py
    return utils.audioToText(filename) 

def synthesize_speech(text, output_filename):
    # Replace with your actual text-to-speech logic from utils.py
    utils.textToAudioFile(text, output_filename) 

def play_audio(filename):
    audio = AudioSegment.from_mp3(filename)
    play(audio)

# --- Main Script ---
input_audio_file = "input4.wav"
output_audio_file = "output4.mp3"

record_audio_vad(input_audio_file)
transcription = transcribe_audio(input_audio_file)
print("Transcription:", transcription)

synthesize_speech(transcription, output_audio_file)
play_audio(output_audio_file)
